<a href="https://colab.research.google.com/github/Tkag0001/Tutorial_deploy_streamlit_vsc_with_colab_server/blob/main/deploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import fastapi
import numpy as np
from pygame.transform import threshold
!pip install pyngrok fastapi uvicorn
!pip install python-multipart
!pip install streamlit

pygame 2.6.0 (SDL 2.28.4, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import os
import threading
import socket
from pyngrok import ngrok, conf
from fastapi import FastAPI
import uvicorn
from fastapi import File, UploadFile, HTTPException
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from fastapi.responses import JSONResponse
import numpy as np

# Load model


In [2]:
labels = ['metal', 'white-glass', 'biological', 'paper', 'brown-glass', 'battery', 'trash', 'cardboard', 'shoes', 'clothes', 'plastic', 'green-glass']

In [3]:
model_path = "./alexnet_model_highest_accuracy.keras"


In [4]:
model_high_accuracy_alexnet = load_model(model_path)

In [6]:
input_shape=(224, 224, 3)

In [5]:
threshold = 0.5 #t lệ ngưỡng xác nhận thành công hay không

# Init deploy process

In [7]:
your_token ="2qyvibdjnlQbKUzmeaQuoivu33Z_25ck2P5Wk5koVmA1PPv9x" # Input your token

In [8]:
import os
# Set your ngrok auth token
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken")
os.system(f"ngrok config add-authtoken {your_token}")

# Initialize FastAPI app
app = FastAPI()

# Start Streamlit server on a specific port
# !nohup streamlit run app.py --server.port 8000 &

# Open an ngrok tunnel to the HTTP server
port = 8080  # Default FastAPI/uvicorn port
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}/\"")

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken
 * ngrok tunnel "https://3e64-2001-ee0-4fee-91d0-743f-4f9d-526c-9c8b.ngrok-free.app" -> "http://127.0.0.1:8080/"


In [9]:
app = FastAPI()
server_thread = None
server = None
# Comment to test deploy streamlit
# Define FastAPI routes
@app.get("/")
async def index():
    hostname = socket.gethostname()
    return {"message": "Hello from Colab!", "hostname": hostname}

@app.get("/hello/{name}")
async def say_hello(name: str):
    return {"message": f"Hello, {name}!"}
# ////////////////////////////////////
# Lấy danh sách tất cả các item
array = [1,2,3]

@app.get("/items/",status_code=404)
async def get_items():
    return {"message": array}

# Lấy thông tin chi tiết của một item theo ID
@app.get("/items/{item_id}")
async def get_item(item_id: int):
    return {"item_id": item_id, "message": "Thông tin item"}


@app.post("/test_post/")
async def post_items(img: UploadFile = File(...)):
    try:
        content = await img.read()
        with open("temp_img.jpg", "wb") as f:
            f.write(content)
            
        # Load và tiền xử lý ảnh
        # Load anh thành dạng có kích thước 224x224
        image = load_img("temp_img.jpg", target_size=input_shape)
        # chuyển image sang dạng ma trận
        image_array = img_to_array(image)
        
        image_array = np.expand_dims(image_array, axis=0)
        # du doan bằng mô hình xem có hoạt động hay không
        prediction = model_high_accuracy_alexnet.predict(image_array)
        labels_index = np.argmax(prediction, axis=1)[0]
        confident = float(np.max(prediction))
        label = labels[labels_index]
        if confident > threshold:
            return {"message": "THÀNH CÔNG", "label": label, "confident": confident}
        else:
            raise HTTPException(
                status_code=400,
                detail={"error": "Xác định không thành công, dưỡng ngưỡng choi phép", "confident": confident}
            )
    except Exception as e:
        # Trả về lỗi JSON trong trường hợp lỗi không mong đợi
        return JSONResponse(
            content={"error": "An unexpected error occurred", "details": str(e)},
            status_code=500
        )
# Start FastAPI server in a separate thread using uvicorn
# def start_uvicorn():
#     uvicorn.run(app, host="0.0.0.0", port=port)
# 
# threading.Thread(target=start_uvicorn, daemon=True).start()
# Start the Uvicorn server
def start_server():
    global server
    config = uvicorn.Config(app, host="127.0.0.1", port=port)
    server = uvicorn.Server(config)
    server.run()

# Stop the Uvicorn server
def stop_server():
    global server
    if server and not server.should_exit:
        server.should_exit = True

# Restart the server
def restart_server():
    global server_thread
    if server_thread and server_thread.is_alive():
        stop_server()
        server_thread.join()
    server_thread = threading.Thread(target=start_server, daemon=True)
    server_thread.start()

# Start the server for the first time
restart_server()